In [52]:
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [53]:
# configuration
SVM_kernel = 'rbf'
cache_size = 8000
svm_params_c = [0.001, 0.1, 100, 10e5]
svm_params_gamma = [10,1,0.1,0.01]
cv_folds = 5
grid_cv_samples = 4000

model_from_files = False

grid_model_file = f'RAPIDS_grid_{SVM_kernel}.pkl'
trained_model_file = f'RAPIDS_model_{SVM_kernel}.pkl'

submission_file = f'RAPIDS_submission_{SVM_kernel}.csv'

In [54]:
train_df = pd.read_csv('../input/digit-recognizer/train.csv')
print(train_df.shape)
test_df = pd.read_csv('../input/digit-recognizer/test.csv')
print(test_df.shape)

(42000, 785)
(28000, 784)


In [55]:
train_df.columns[0:10]

Index(['label', 'pixel0', 'pixel1', 'pixel2', 'pixel3', 'pixel4', 'pixel5',
       'pixel6', 'pixel7', 'pixel8'],
      dtype='object')

In [56]:
# lets checkout some images
# plt.figure(figsize=(12, 10))
# for i in range(1, 5):
#     plt.subplot(1, 4, i)
#     plt.imshow(train_df.iloc[i-1, 1:].values.reshape(28, 28), cmap='gray')
#     plt.title(f'plot for {train_df.iloc[i-1, 0]}')
# plt.show()

In [57]:
# sns.countplot(train_df['label'])
# plt.show()

In [58]:
from IPython.core.display import display, HTML
HTML('''<script> </script> <form action="javascript:IPython.notebook.execute_cells_above()"><input type="submit" id="toggleButton" value="Run all above Cells"></form>''')

### Modelling

In [59]:
from cuml.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV

import time
import pickle

In [60]:
# if not model_from_files:
#     steps = [('scaler', MinMaxScaler()), ('SVM', SVC(kernel=SVM_kernel, cache_size=cache_size))]
#     pipeline = Pipeline(steps)

#     params = {'SVM__C':svm_params_c, 'SVM__gamma':svm_params_gamma}
#     grid = GridSearchCV(pipeline,
#                         param_grid = params,
#                         scoring='accuracy',
#                         cv = cv_folds)

#     t_start = time.time()

#     grid.fit(train_df.iloc[:grid_cv_samples, 1:], train_df.iloc[:grid_cv_samples, 0])

#     print("Elapsed Time", time.time()-t_start)

#     # save the model
#     pickle.dump(grid, open(grid_model_file, 'wb'))

In [61]:
# import sklearn
# sklearn.metrics.SCORERS.keys()

In [62]:
# if model_from_files:
#     grid = pickle.load(open(grid_model_file, 'rb'))

In [63]:
# print(f"Best score = {grid.best_score_}")
# print(f"Best params = {grid.best_params_}")

In [16]:
from IPython.core.display import display, HTML
HTML('''<script> </script> <form action="javascript:IPython.notebook.execute_cells_above()"><input type="submit" id="toggleButton" value="Run all above Cells"></form>''')

### Acutal training

In [66]:
if not model_from_files:
    steps = [('scaler', MinMaxScaler()), ('SVM', SVC(kernel=SVM_kernel, cache_size=cache_size, 
                                                     gamma=0.01, 
                                                     C=100))]
    scaler = MinMaxScaler()
    X_train = train_df.iloc[:, 1:]
    X_train = scaler.fit_transform(X_train)

    t_start = time.time()

    model = SVC(kernel=SVM_kernel, cache_size=cache_size, gamma=0.01, C=100)
    # train on entire train data set
    model.fit(X_train, train_df.iloc[:, 0])

    print("Elapsed Time", time.time()-t_start)

    # save the model
    pickle.dump(pipeline, open(trained_model_file, 'wb'))

TypeError: not all arguments converted during string formatting

In [11]:
if model_from_files:
    pipeline = pickle.load(open(trained_model_file, 'rb'))

In [18]:
from IPython.core.display import display, HTML
HTML('''<script> </script> <form action="javascript:IPython.notebook.execute_cells_above()"><input type="submit" id="toggleButton" value="Run all above Cells"></form>''')

### Derive test results

In [45]:
submission = pd.DataFrame()
submission['ImageId'] = test_df.index + 1
submission.head()

,ImageId
0,1
1,2
2,3
3,4
4,5


In [46]:
submission['Label'] = pipeline.predict(test_df)

In [47]:
submission.to_csv(submission_file, index=False)